In [218]:
import pandas as pd
import math
import sys
from statistics import mean

In [219]:
df = pd.read_csv("chocolate/0.csv", usecols=['ch0', 'ch1', 'ch2', 'ch3'])
# print(df)
# pct_change = df.pct_change()
# print(pch_change)

In [220]:
# manual input
high_voltages = [4.8059039E-05,0.000175995752,4.0310435E-05,3.3339486E-05]
low_voltages  = [2.3650005E-05, 0.000142542645, 1.8811785E-05, 3.214926E-06]

total_rows = df.shape[0]
middle = (low_voltages[0] + high_voltages[0]) / 2.0

picked_prev = False
changing_points = []

for i in range(total_rows):
    picked_curr = df.loc[i][0] < middle
    if bool(picked_prev) ^ bool(picked_curr):
        changing_points.append(i)
    picked_prev = picked_curr

print(changing_points)

[47, 161, 162, 166, 231, 344, 347, 350, 420, 532, 536, 539, 612, 729, 731, 732, 796, 900, 903, 904, 964, 1039, 1041, 1042, 1101, 1173, 1174, 1179, 1180, 1181, 1182, 1183, 1246]


In [222]:
sample_size = 31
sample_radius = math.floor(sample_size / 2)
indexes = []
fluctuation = []
changing_points.append(sys.float_info.max)

for i in range(1, len(changing_points)):
    changing_point = changing_points[i - 1]
    fluctuation.append(changing_point)
    if changing_points[i] - changing_point > sample_radius: 
        changing_point = math.floor(mean(fluctuation))
        fluctuation = []
        # indexes.append(changing_point)
        indexes.append([changing_point - sample_radius, changing_point + sample_radius])

print(indexes)       

[[32, 62], [148, 178], [216, 246], [332, 362], [405, 435], [520, 550], [597, 627], [715, 745], [781, 811], [887, 917], [949, 979], [1025, 1055], [1086, 1116], [1163, 1193], [1231, 1261]]


In [243]:
# a = pd.DataFrame(pd.DataFrame(df.loc[32:62]))
# b = pd.DataFrame(df.loc[148:178])
# print(a)
# print(b)
# c = a.append(b)
# print(c)

parsed = pd.DataFrame()

for index in indexes:
    parsed = parsed.append(pd.DataFrame(df.loc[index[0]:index[1]]))

parsed.to_csv('out0.csv')